In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_digits
import sys
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np




#导入数据
def read_and_decode(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.cast(img,tf.float64)

    img = tf.reshape(img, [48, 24, 3])
    img=tf.split(img,3,2)[0]
    label = tf.cast(features['label'], tf.int32)
    img_batch, label_batch = tf.train.shuffle_batch([img, label],
                                                batch_size=64, capacity=25000,
                                                min_after_dequeue=10000)
    
    return img_batch, label_batch

def read_and_decode1(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.cast(img,tf.float64)

    img = tf.reshape(img, [48, 24, 3])
    img=tf.split(img,3,2)[0]
    label = tf.cast(features['label'], tf.int32) 
    return img_batch, label_batch




def dr(a):
    s = list()
    for index, i in enumerate(a):
        s.append(list())
        for n in range(34):
            if n == i:
                s[index].append(1)
            else:
                s[index].append(0)            
    return s


"""
权重初始化,
初始化为一个接近0的很小的正数

"""


       
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)


#卷积:步长为1,0边距
def juanji(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = 'VALID')


def chihua(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
#MNIST数据输入


#第一层 卷积层
x = tf.placeholder(tf.float32,[None, 48,24,1])

#最后一维代表通道，如果是rgb则为3 
x_image = tf.reshape(x, [-1, 48, 24, 1])
# 第一二参数值得卷积核尺寸大小，即patch，第三个参数是图像通道数，第四个参数是卷积核的数目，代表会出现多少个卷积特征图像;  
W_1 = weight_variable([5, 5, 1, 32])
# 对于每一个卷积核都有一个对应的偏置量。
b_1 = bias_variable([32])
# 图片乘以卷积核，并加上偏执量，
h_j1 = tf.nn.relu(juanji(x_image, W_1) + b_1)
h_c1 = chihua(h_j1)

In [3]:
#第二层 卷积层
# 32通道卷积，卷积出64个特征 
W_2 = weight_variable([5, 5, 32, 64])
# 64个偏执数据 
b_2 = bias_variable([64])
h_j2 = tf.nn.relu(juanji(h_c1, W_2) + b_2)
h_c2 = chihua(h_j2)

In [4]:
#第三层 全连接层
W_f1 = weight_variable([9 * 3 * 64, 1024])
b_f1 = bias_variable([1024])
h_c2_flat = tf.reshape(h_c2, [-1, 9 * 3 * 64])
h_f1 = tf.nn.relu(tf.matmul(h_c2_flat, W_f1) + b_f1)
keep_prob = tf.placeholder("float")
h_f1_drop = tf.nn.dropout(h_f1, keep_prob)




In [5]:
#第四层 Softmax输出层

W_f2 = weight_variable([1024, 34])
b_f2 = bias_variable([34])
logits = tf.matmul(h_f1_drop, W_f2) + b_f2
#Y = tf.nn.softmax(tf.matmul(h_f1_drop, W_f2) + b_f2)

In [6]:
y_ = tf.placeholder("float32", [None, 34])
#cross_entropy = -tf.reduce_sum(y_ * tf.log(Y + 1e-10)) # 定义交叉熵为loss函数    
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits)
train_step = tf.train.AdamOptimizer().minimize(cross_entropy) # 调用优化
saver = tf.train.Saver()
Y = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(y_,1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
img_batch ,label_batch = read_and_decode("train.tfrecords")
img_v_batch ,label_v_batch = read_and_decode1("validation.tfrecords")

In [7]:
#保存模型
#保存最优的三个
#幷记录每100次的正确率
saver=tf.train.Saver(max_to_keep=3)
max_train_accuracy=0
f=open('model/acc.txt','w')

In [9]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./logs', graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    #训练阶段
    for i in range(5000):
        img, label = sess.run([img_batch, label_batch]) 
        #print(label[0])
        label = dr(label)
        #plt.imshow(img[0].reshape(48,24),cmap="gray")
        #plt.show()
        
        if i%100 == 0:  
            train_accuracy = accuracy.eval(feed_dict={x:img, y_: label, keep_prob: 1.0})  
            print("step %d, training accuracy %g"%(i, train_accuracy)) 
            f.write(str(i+1)+', train_accuracy: '+str(train_accuracy)+'\n')
            if train_accuracy>max_train_accuracy:
                max_train_accuracy = train_accuracy
                saver.save(sess,'model/mnist.ckpt',global_step=i+1)
        train_step.run(feed_dict={x:img, y_: label, keep_prob: 0.5}) 
    f.close()
    
    #模型的恢复
    model_file=tf.train.latest_checkpoint('model/')
    saver.restore(sess,model_file)
    #验证阶段
    img_v, label_v = sess.run([img_v_batch, label_v_batch])
    label_v = dr(label_v)
    print("validation accuracy %g" %accuracy.eval(session = sess,feed_dict = {x:img_v, y_:label_v,keep_prob:1.0}))
    
    
    coord.request_stop()
    coord.join(threads)

step 0, training accuracy 0.03125
step 100, training accuracy 0.140625
step 200, training accuracy 0.078125
step 300, training accuracy 0.0625
step 400, training accuracy 0.078125
step 500, training accuracy 0.0625
step 600, training accuracy 0.046875
step 700, training accuracy 0.046875
step 800, training accuracy 0.15625
step 900, training accuracy 0.03125
step 1000, training accuracy 0.09375
step 1100, training accuracy 0.0625
step 1200, training accuracy 0.0625
step 1300, training accuracy 0.109375
step 1400, training accuracy 0.078125
step 1500, training accuracy 0.0625
step 1600, training accuracy 0.078125
step 1700, training accuracy 0.078125
step 1800, training accuracy 0.109375
step 1900, training accuracy 0.078125
step 2000, training accuracy 0.109375
step 2100, training accuracy 0.125
step 2200, training accuracy 0.125
step 2300, training accuracy 0.1875
step 2400, training accuracy 0.140625
step 2500, training accuracy 0.40625
step 2600, training accuracy 0.40625
step 2700,